In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("C7").getOrCreate()
!netstat -anp |grep 4040

tcp6       0      0 :::4040                 :::*                    LISTEN      14404/java          


In [2]:
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("/root/golive/Spark-The-Definitive-Guide/data/retail-data/all/*.csv")\
.coalesce(5)

df.show(5)
df.createOrReplaceTempView("t1")

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [3]:
# BASIC Aggregation Functions 
# Count - Aggregate Functions 

from pyspark.sql.functions import col 
sql = """
select count(*) as countall from t1 
"""
spark.sql(sql).show()

df.selectExpr("count(StockCode) as countall").show()

+--------+
|countall|
+--------+
|  541909|
+--------+

+--------+
|countall|
+--------+
|  541909|
+--------+



In [4]:
df.selectExpr("count(StockCode) as countall").explain()

== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[count(StockCode#17)])
+- Exchange SinglePartition, true, [id=#115]
   +- *(1) HashAggregate(keys=[], functions=[partial_count(StockCode#17)])
      +- Coalesce 5
         +- FileScan csv [StockCode#17] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/root/golive/Spark-The-Definitive-Guide/data/retail-data/all/online-retail..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<StockCode:string>




In [5]:
# CountDistinct Aggregate Functions 
from pyspark.sql.functions import countDistinct
df.select(countDistinct("StockCode").alias("count_distinct")).show()
df.select(countDistinct("StockCode").alias("count_distinct")).explain()

+--------------+
|count_distinct|
+--------------+
|          4070|
+--------------+

== Physical Plan ==
*(3) HashAggregate(keys=[], functions=[count(distinct StockCode#17)])
+- Exchange SinglePartition, true, [id=#196]
   +- *(2) HashAggregate(keys=[], functions=[partial_count(distinct StockCode#17)])
      +- *(2) HashAggregate(keys=[StockCode#17], functions=[])
         +- Exchange hashpartitioning(StockCode#17, 200), true, [id=#191]
            +- *(1) HashAggregate(keys=[StockCode#17], functions=[])
               +- Coalesce 5
                  +- FileScan csv [StockCode#17] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/root/golive/Spark-The-Definitive-Guide/data/retail-data/all/online-retail..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<StockCode:string>




In [6]:
sql = """
select count(distinct(StockCode)) from t1"""
spark.sql(sql).show()
spark.sql(sql).explain()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+

== Physical Plan ==
*(3) HashAggregate(keys=[], functions=[count(distinct StockCode#17)])
+- Exchange SinglePartition, true, [id=#277]
   +- *(2) HashAggregate(keys=[], functions=[partial_count(distinct StockCode#17)])
      +- *(2) HashAggregate(keys=[StockCode#17], functions=[])
         +- Exchange hashpartitioning(StockCode#17, 200), true, [id=#272]
            +- *(1) HashAggregate(keys=[StockCode#17], functions=[])
               +- Coalesce 5
                  +- FileScan csv [StockCode#17] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/root/golive/Spark-The-Definitive-Guide/data/retail-data/all/online-retail..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<StockCode:string>




In [7]:
from pyspark.sql.functions import approxCountDistinct , approx_count_distinct
sql = """
select approx_count_distinct(StockCode,0.1) from t1"""
spark.sql(sql).show()
spark.sql(sql).explain()

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+

== Physical Plan ==
HashAggregate(keys=[], functions=[approx_count_distinct(StockCode#17, 0.1, 0, 0)])
+- Exchange SinglePartition, true, [id=#315]
   +- HashAggregate(keys=[], functions=[partial_approx_count_distinct(StockCode#17, 0.1, 0, 0)])
      +- Coalesce 5
         +- FileScan csv [StockCode#17] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/root/golive/Spark-The-Definitive-Guide/data/retail-data/all/online-retail..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<StockCode:string>




In [8]:
df.select(approx_count_distinct(col("StockCode"),0.1)).show()

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



In [9]:
# first , last
from pyspark.sql.functions import first , last 
df.select(first(col("StockCode")), last(col("StockCode"))).show()

sql = """ select first(stockcode) , last(stockcode) from t1"""
spark.sql(sql).show()

+-----------------------+----------------------+
|first(StockCode, false)|last(StockCode, false)|
+-----------------------+----------------------+
|                 85123A|                 22138|
+-----------------------+----------------------+

+-----------------------+----------------------+
|first(stockcode, false)|last(stockcode, false)|
+-----------------------+----------------------+
|                 85123A|                 22138|
+-----------------------+----------------------+



In [10]:
from pyspark.sql.functions import min, max 
df.select(min(col("StockCode")), max(col("StockCode"))).show()

sql = """ select min(stockcode) , max(stockcode) from t1"""
spark.sql(sql).show()

+--------------+--------------+
|min(StockCode)|max(StockCode)|
+--------------+--------------+
|         10002|             m|
+--------------+--------------+

+--------------+--------------+
|min(stockcode)|max(stockcode)|
+--------------+--------------+
|         10002|             m|
+--------------+--------------+



In [11]:
# functions like sum , average , mean works in similar fashion 

In [12]:
sql = """SELECT var_pop( Quantity) var_pop ,var_samp( Quantity) var_samp , stddev_pop( Quantity) std_pop,
stddev_samp( Quantity)  std_samp
FROM t1"""
spark.sql(sql).show()

from pyspark.sql.functions import var_samp, var_pop , stddev_samp, stddev_pop
df.select(var_pop(col("Quantity"))\
          ,var_samp(col("Quantity"))\
          ,stddev_pop(col("Quantity"))\
          ,stddev_samp(col("Quantity"))\
         ).show()



+-----------------+-----------------+------------------+------------------+
|          var_pop|         var_samp|           std_pop|          std_samp|
+-----------------+-----------------+------------------+------------------+
|47559.30364660923|47559.39140929892|218.08095663447835|218.08115785023455|
+-----------------+-----------------+------------------+------------------+

+-----------------+------------------+--------------------+---------------------+
|var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+-----------------+------------------+--------------------+---------------------+
|47559.30364660923| 47559.39140929892|  218.08095663447835|   218.08115785023455|
+-----------------+------------------+--------------------+---------------------+



In [13]:
# SKEWNESS , kurtosis

sql = """
SELECT skewness( Quantity), kurtosis( Quantity) FROM t1
"""
spark.sql(sql).show()

# COVARIANCE , CORRELATION 


sql = """
SELECT corr( InvoiceNo, Quantity) as correlation , covar_samp( InvoiceNo, Quantity) as cv_samp, 
covar_pop( InvoiceNo, Quantity) as cv_pop FROM t1
"""
spark.sql(sql).show()


+----------------------------------+----------------------------------+
|skewness(CAST(Quantity AS DOUBLE))|kurtosis(CAST(Quantity AS DOUBLE))|
+----------------------------------+----------------------------------+
|              -0.26407557610528376|                119768.05495530753|
+----------------------------------+----------------------------------+

+--------------------+------------------+------------------+
|         correlation|           cv_samp|            cv_pop|
+--------------------+------------------+------------------+
|4.912186085640497E-4|1052.7280543915997|1052.7260778754955|
+--------------------+------------------+------------------+



In [14]:
from pyspark.sql.functions import collect_list,collect_set
df.printSchema()
df.select(collect_list(col("Country"))\
          ,collect_set(col("Country"))).show(1)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

+---------------------+--------------------+
|collect_list(Country)|collect_set(Country)|
+---------------------+--------------------+
| [United Kingdom, ...|[Portugal, Italy,...|
+---------------------+--------------------+



In [15]:
# Group by InvoiceNo & CustomerID - Aggregate Function : Count 
from pyspark.sql.functions import count
sql = """
select InvoiceNo, CustomerID, Count(*) from t1 group by InvoiceNo, CustomerID
"""
spark.sql(sql).show(3)

df.groupBy(col("InvoiceNo"),col("CustomerID"))\
.agg(count("InvoiceNo").alias("count_Invoice"))\
.show(3)



+---------+----------+--------+
|InvoiceNo|CustomerID|count(1)|
+---------+----------+--------+
|   536846|     14573|      76|
|   537026|     12395|      12|
|   537883|     14437|       5|
+---------+----------+--------+
only showing top 3 rows

+---------+----------+-------------+
|InvoiceNo|CustomerID|count_Invoice|
+---------+----------+-------------+
|   536846|     14573|           76|
|   537026|     12395|           12|
|   537883|     14437|            5|
+---------+----------+-------------+
only showing top 3 rows



In [16]:
# group by expression 
from pyspark.sql.functions import expr

df.groupBy(col("InvoiceNo"),col("CustomerID"))\
.agg(count("InvoiceNo").alias("count_Invoice"), expr("count(InvoiceNo)").alias("expr_count_inv"))\
.show(3)


df.groupBy(col("InvoiceNo"),col("CustomerID"))\
.agg(count("InvoiceNo").alias("count_Invoice"), expr("count(InvoiceNo)").alias("expr_count_inv")).explain(True)

+---------+----------+-------------+--------------+
|InvoiceNo|CustomerID|count_Invoice|expr_count_inv|
+---------+----------+-------------+--------------+
|   536846|     14573|           76|            76|
|   537026|     12395|           12|            12|
|   537883|     14437|            5|             5|
+---------+----------+-------------+--------------+
only showing top 3 rows

== Parsed Logical Plan ==
'Aggregate ['InvoiceNo, 'CustomerID], [unresolvedalias('InvoiceNo, None), unresolvedalias('CustomerID, None), count('InvoiceNo) AS count_Invoice#1815, 'count('InvoiceNo) AS expr_count_inv#1816]
+- Repartition 5, false
   +- Relation[InvoiceNo#16,StockCode#17,Description#18,Quantity#19,InvoiceDate#20,UnitPrice#21,CustomerID#22,Country#23] csv

== Analyzed Logical Plan ==
InvoiceNo: string, CustomerID: int, count_Invoice: bigint, expr_count_inv: bigint
Aggregate [InvoiceNo#16, CustomerID#22], [InvoiceNo#16, CustomerID#22, count(InvoiceNo#16) AS count_Invoice#1815L, count(InvoiceNo

In [17]:
# Group by using MAP ( Not sure what is MAP here though)
from pyspark.sql.functions import stddev_pop
sql = """
select InvoiceNO, avg(Quantity), stddev_pop(Quantity)
from t1
Group by InvoiceNO"""

spark.sql(sql).show(3)

+---------+------------------+------------------------------------+
|InvoiceNO|     avg(Quantity)|stddev_pop(CAST(Quantity AS DOUBLE))|
+---------+------------------+------------------------------------+
|   536596|               1.5|                  1.1180339887498947|
|   536938|33.142857142857146|                  20.698023172885524|
|   537252|              31.0|                                 0.0|
+---------+------------------+------------------------------------+
only showing top 3 rows



In [18]:
from pyspark.sql.functions import avg , stddev_pop
df.groupBy(col("InvoiceNO"))\
.agg(avg("Quantity").alias("avg_quantity"), stddev_pop("Quantity").alias("std_dev_quant"))\
.show(3)

+---------+------------------+------------------+
|InvoiceNO|      avg_quantity|     std_dev_quant|
+---------+------------------+------------------+
|   536596|               1.5|1.1180339887498947|
|   536938|33.142857142857146|20.698023172885524|
|   537252|              31.0|               0.0|
+---------+------------------+------------------+
only showing top 3 rows



# Window Functions 

In [30]:
# spark supports 3 kinds of window functions :
# 1. Aggregate function 
# 2. Ranking  function
# 3. Analytical function 
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
from pyspark.sql.functions import to_date
df.show(2)
dfwithdate =df.withColumn("date",to_date("InvoiceDate","MM/d/yyyy H:mm"))
dfwithdate.select("*").show(2,False)

dfwithdate.where("date is null").show()


+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 2 rows

+---------+---------+----------------------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|Description                       |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |date      |
+---------+---------+----------------------------------+--------+--------------+---------+----------+

In [31]:
# step 1 : create window specification 
from pyspark.sql.window import Window
from pyspark.sql.functions import desc
windowspec = Window\
.partitionBy(col("CustomerID"), col("date"))\
.orderBy(col("Quantity").desc())\
.rowsBetween(Window.unboundedPreceding,Window.currentRow)



In [32]:
# step 2 - Use one of Aggregate , Ranking or Analytical function on the window specification 
#Keyword - over 
from pyspark.sql.functions import max , min
max_quant_window = max(col("Quantity")).over(windowspec)

# Aggregate Function OVER 

#dfwithdate.withColumn("max_quant_window",max(col("Quantity")).over(windowspec))\
#.show()

#df_new.show(2)
#maxPurchaseQuantity = max(col("Quantity")).over( windowSpec)
#df_new.orderBy(col("max_quant_window").desc()).show(2)




In [34]:
# Ranking function Over WindowSpecification 

from pyspark.sql.functions import rank,dense_rank
purchaseDenseRank = dense_rank().over(windowspec)
type(purchaseDenseRank)
purchaseRank = rank().over(windowspec)

#dfwithdate.printSchema()
dfwithdate.where("CustomerID is not null ")\
.orderBy(col("CustomerID").desc())\
.select(col("CustomerID"), col("date"), col("quantity")\
        ,max_quant_window.alias("max_quantity_window")\
        ,purchaseDenseRank.alias("Dense_Rank_window")\
        ,purchaseRank.alias("Rank_window"))\
.show(5)


+----------+----------+--------+-------------------+-----------------+-----------+
|CustomerID|      date|quantity|max_quantity_window|Dense_Rank_window|Rank_window|
+----------+----------+--------+-------------------+-----------------+-----------+
|     18245|2010-12-19|      24|                 24|                1|          1|
|     18245|2010-12-19|      24|                 24|                1|          1|
|     18245|2010-12-19|      24|                 24|                1|          1|
|     18245|2010-12-19|      12|                 24|                2|          4|
|     18245|2010-12-19|      12|                 24|                2|          4|
+----------+----------+--------+-------------------+-----------------+-----------+
only showing top 5 rows



In [44]:
dfwithdate.createOrReplaceTempView("t_dfwithdate")
sql = """
select  customerID , 
date ,
quantity ,
max( Quantity) OVER (PARTITION BY CustomerId, date 
                    ORDER BY Quantity DESC NULLS LAST 
                    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as maxPurchase,
rank(Quantity) OVER (PARTITION BY CustomerId, date 
                    ORDER BY Quantity DESC NULLS LAST 
                    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as rank_window ,
                    
dense_rank(Quantity) OVER (PARTITION BY CustomerId, date 
                    ORDER BY Quantity DESC NULLS LAST 
                    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as Dense_rank_window

from t_dfwithdate
where CustomerID is not null 
order by CustomerID desc 

"""

spark.sql(sql).show(5)

+----------+----------+--------+-----------+-----------+-----------------+
|customerID|      date|quantity|maxPurchase|rank_window|Dense_rank_window|
+----------+----------+--------+-----------+-----------+-----------------+
|     18287|2011-05-22|      30|         60|          4|                4|
|     18287|2011-05-22|      48|         60|          2|                2|
|     18287|2011-05-22|      36|         60|          3|                3|
|     18287|2011-05-22|      60|         60|          1|                1|
|     18287|2011-05-22|      24|         60|          6|                5|
+----------+----------+--------+-----------+-----------+-----------------+
only showing top 5 rows



In [52]:
# Grouping Sets 

dfNoNull = dfwithdate.drop() 
dfNoNull.createOrReplaceTempView("dfNoNull")

sql = """
SELECT CustomerId, stockCode, sum( Quantity) 
FROM dfNoNull 
GROUP BY customerId, stockCode
order by customerid desc , stockcode desc 


"""
spark.sql(sql).show(5)


sql = """
SELECT CustomerId, stockCode, sum( Quantity) 
FROM dfNoNull 
GROUP BY customerId, stockCode grouping sets(customerId, stockCode , ())
order by customerid desc , stockcode desc 


"""
spark.sql(sql).show(5)

+----------+---------+-------------+
|CustomerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
+----------+---------+-------------+
only showing top 5 rows

+----------+---------+-------------+
|customerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|     null|         1586|
|     18283|     null|         1397|
|     18282|     null|           98|
|     18281|     null|           54|
|     18280|     null|           45|
+----------+---------+-------------+
only showing top 5 rows



In [ ]:
# https://oracle-base.com/articles/misc/rollup-cube-grouping-functions-and-grouping-sets